# <span style="color:purple">Syntax preprocessing</span>

This tutorial describes the tools that are needed to convert the morphologically analysed Estonian text into the format on which syntactic parsing can be applied. 
Note that Estnltk supports multiple syntactic parsers, and each parser typically has multiple models, so preprocessing setups are varying and there is no single preprocessing setup used in all occasions.
Here we introduce commonly used preprocessing steps, and parsers' tutorials give details about which preprocessing is needed.

## MorphExtendedTagger and 'morph_extended' layer

Layer 'morph_extended' is an extended version of Vabamorf's 'morph_analysis' layer, which has been augmented with additional morpho-syntactic information, such as pronoun type, verb form and verb subcategorization information. 
It is used as an input by Visl CG3-based syntactic analysis, and also by MaltParser's, UDPipe's, and stanza's models.

You can create 'morph_extended' layer directly via default resolver:

In [1]:
from estnltk import Text

text = Text('laulma hüplev tantsija').tag_layer('morph_extended')

text['morph_extended']

Layer(name='morph_extended', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'punctuation_type', 'pronoun_type', 'letter_case', 'fin', 'verb_extension_suffix', 'subcat'), spans=SL[Span('laulma', [{'normalized_text': 'laulma', 'lemma': 'laulma', 'root': 'laul', 'root_tokens': ['laul'], 'ending': 'ma', 'clitic': '', 'form': 'mod sup ps ill', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': False, 'verb_extension_suffix': [], 'subcat': ['NGP-P', 'El', 'All']}, {'normalized_text': 'laulma', 'lemma': 'laulma', 'root': 'laul', 'root_tokens': ['laul'], 'ending': 'ma', 'clitic': '', 'form': 'aux sup ps ill', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': False, 'verb_extension_suffix': [], 'subcat': ['NGP-P', 'El', 'All']}, {'normalized_text': 'laulma', 'lemma': 'laulma', 'root': 'laul', 'root_tokens': ['laul'], 'ending': 'ma', 'clitic': '', 'form': 'main sup ps ill', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': False, 'verb_extension_suffix': [], 'subcat': ['NGP-P', 'El', 'All']}]),
Span('hüplev', [{'normalized_text': 'hüplev', 'lemma': 'hüplev', 'root': 'hüplev', 'root_tokens': ['hüplev'], 'ending': '0', 'clitic': '', 'form': 'pos sg nom', 'partofspeech': 'A', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': None, 'verb_extension_suffix': [], 'subcat': None}]),
Span('tantsija', [{'normalized_text': 'tantsija', 'lemma': 'tantsija', 'root': 'tantsija', 'root_tokens': ['tantsija'], 'ending': '0', 'clitic': '', 'form': 'com sg nom', 'partofspeech': 'S', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': None, 'verb_extension_suffix': [], 'subcat': None}])])

Note that the 'morph_extended' layer is _ambiguous_ as the conversion process does not involve disambiguation.

The 'morph_extended' layer is created by **MorphExtendedTagger**. 
Taggers and retaggers that are used in the process are described here:

|tagger or retagger|source attributes|target attributes|output values|
|------------------|-----------------|-----------------|------|
|MorphToSyntaxMorphRetagger |_Vabamorf's attributes_ | _Vabamorf's attributes_ | partofspeech & form tags from here: https://www.cl.ut.ee/korpused/morfliides/seletus/ |
|PunctuationTypeRetagger | partofspeech, root|punctuation_type| ```None```, 'Fst', 'Com', 'Col', 'Ell', 'Els', 'Scl', 'Int', 'Exc', 'Dsd', 'Dsh', 'Opr', 'Cpr', 'Quo', 'Oqu', 'Cqu', 'Grt', 'Sml', 'Osq', 'Csq', 'Sla', 'crd' |
|PronounTypeRetagger|root, ending, clitic, partofspeech|pronoun_type| ```None```, ('det',), ('pers ps3',), ('pos', 'det', 'refl'), ... [(based on this lexicon)](https://github.com/estnltk/estnltk/blob/main/estnltk/estnltk/taggers/standard/syntax/preprocessing/rules_files/pronouns.csv) |
|LetterCaseRetagger|word_text|cap|```None```, 'cap'|
|RemoveAdpositionAnalysesRetagger|partofspeech, form|&nbsp;|&nbsp;|
|FiniteFormRetagger|partofspeech, form|fin|```None```, ```True```, ```False```|
|VerbExtensionSuffixRetagger|root|verb_extension_suffix|```None```,'tud','nud','mine','nu','tu','du','v','tav','dav','mata','ja'|
|SubcatRetagger|root, partofspeech, form|subcat|```None```, 'Intr', 'Part', 'gen', ...[(based on this lexicon)](https://github.com/estnltk/estnltk/blob/main/estnltk/estnltk/taggers/standard/syntax/preprocessing/rules_files/abileksikon06utf.lx) |

In the following, we'll exemplify the usage of these taggers in detail.

### MorphToSyntaxMorphRetagger
MorphToSyntaxMorphRetagger is the first preprocessing step, which converts morphological categories from Vabamorf's format to the CG format described here: https://www.cl.ut.ee/korpused/morfliides/seletus/ .
( For a comparison of different Estonian morpho-syntactic category systems, please see the document https://cl.ut.ee/ressursid/morfo-systeemid/ )

MorphToSyntaxMorphRetagger can be used both as a tagger (to create a new layer), and as a retagger (to change an existing layer).

Usage example (creating a new layer):

In [2]:
from estnltk.taggers.standard.syntax.preprocessing.morph_to_syntax_morph_retagger import MorphToSyntaxMorphRetagger
base_morph_converter = MorphToSyntaxMorphRetagger()

text = Text('Kumb, sina või mina?').tag_layer('morph_analysis')
base_morph_converter.tag(text)
text['morph_extended']

Layer(name='morph_extended', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Kumb', [{'normalized_text': 'Kumb', 'lemma': 'kumb', 'root': 'kumb', 'root_tokens': ['kumb'], 'ending': '0', 'clitic': '', 'form': 'sg nom', 'partofspeech': 'P'}]),
Span(',', [{'normalized_text': ',', 'lemma': ',', 'root': ',', 'root_tokens': [','], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}]),
Span('sina', [{'normalized_text': 'sina', 'lemma': 'sina', 'root': 'sina', 'root_tokens': ['sina'], 'ending': '0', 'clitic': '', 'form': 'sg nom', 'partofspeech': 'P'}]),
Span('või', [{'normalized_text': 'või', 'lemma': 'või', 'root': 'või', 'root_tokens': ['või'], 'ending': '0', 'clitic': '', 'form': 'sub crd', 'partofspeech': 'J'}]),
Span('mina', [{'normalized_text': 'mina', 'lemma': 'mina', 'root': 'mina', 'root_tokens': ['mina'], 'ending': '0', 'clitic': '', 'form': 'com sg nom', 'partofspeech': 'S'}]),
Span('?', [{'normalized_text': '?', 'lemma': '?', 'root': '?', 'root_tokens': ['?'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

### PunctuationTypeRetagger	

PunctuationTypeRetagger	adds information about punctuation types:

In [3]:
from estnltk.taggers.standard.syntax.preprocessing.punctuation_type_retagger import PunctuationTypeRetagger

retagger = PunctuationTypeRetagger()

text = Text('Kumb, sina või mina?').tag_layer('morph_analysis')
base_morph_converter.tag(text)

retagger.retag(text)
text['morph_extended']

Layer(name='morph_extended', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'punctuation_type'), spans=SL[Span('Kumb', [{'normalized_text': 'Kumb', 'lemma': 'kumb', 'root': 'kumb', 'root_tokens': ['kumb'], 'ending': '0', 'clitic': '', 'form': 'sg nom', 'partofspeech': 'P', 'punctuation_type': None}]),
Span(',', [{'normalized_text': ',', 'lemma': ',', 'root': ',', 'root_tokens': [','], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z', 'punctuation_type': 'Com'}]),
Span('sina', [{'normalized_text': 'sina', 'lemma': 'sina', 'root': 'sina', 'root_tokens': ['sina'], 'ending': '0', 'clitic': '', 'form': 'sg nom', 'partofspeech': 'P', 'punctuation_type': None}]),
Span('või', [{'normalized_text': 'või', 'lemma': 'või', 'root': 'või', 'root_tokens': ['või'], 'ending': '0', 'clitic': '', 'form': 'sub crd', 'partofspeech': 'J', 'punctuation_type': None}]),
Span('mina', [{'normalized_text': 'mina', 'lemma': 'mina', 'root': 'mina', 'root_tokens': ['mina'], 'ending': '0', 'clitic': '', 'form': 'com sg nom', 'partofspeech': 'S', 'punctuation_type': None}]),
Span('?', [{'normalized_text': '?', 'lemma': '?', 'root': '?', 'root_tokens': ['?'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z', 'punctuation_type': 'Int'}])])

### PronounTypeRetagger

PronounTypeRetagger adds information about pronoun types, based on [this lexicon](https://github.com/estnltk/estnltk/blob/main/estnltk/estnltk/taggers/standard/syntax/preprocessing/rules_files/pronouns.csv).  Usage example:

In [4]:
from estnltk.taggers import PronounTypeRetagger

retagger = PronounTypeRetagger()

text = Text('Kumb, sina või mina?').tag_layer('morph_analysis')
base_morph_converter.tag(text)

retagger.retag(text)
text['morph_extended']

Layer(name='morph_extended', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'pronoun_type'), spans=SL[Span('Kumb', [{'normalized_text': 'Kumb', 'lemma': 'kumb', 'root': 'kumb', 'root_tokens': ['kumb'], 'ending': '0', 'clitic': '', 'form': 'sg nom', 'partofspeech': 'P', 'pronoun_type': ('rel',)}]),
Span(',', [{'normalized_text': ',', 'lemma': ',', 'root': ',', 'root_tokens': [','], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z', 'pronoun_type': None}]),
Span('sina', [{'normalized_text': 'sina', 'lemma': 'sina', 'root': 'sina', 'root_tokens': ['sina'], 'ending': '0', 'clitic': '', 'form': 'sg nom', 'partofspeech': 'P', 'pronoun_type': ('ps2',)}]),
Span('või', [{'normalized_text': 'või', 'lemma': 'või', 'root': 'või', 'root_tokens': ['või'], 'ending': '0', 'clitic': '', 'form': 'sub crd', 'partofspeech': 'J', 'pronoun_type': None}]),
Span('mina', [{'normalized_text': 'mina', 'lemma': 'mina', 'root': 'mina', 'root_tokens': ['mina'], 'ending': '0', 'clitic': '', 'form': 'com sg nom', 'partofspeech': 'S', 'pronoun_type': None}]),
Span('?', [{'normalized_text': '?', 'lemma': '?', 'root': '?', 'root_tokens': ['?'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z', 'pronoun_type': None}])])

### LetterCaseRetagger

LetterCaseRetagger simply marks words starting with a capital letter:

In [5]:
from estnltk.taggers.standard.syntax.preprocessing.letter_case_retagger import LetterCaseRetagger

retagger = LetterCaseRetagger()

text = Text('Kumb, Sina või mina?').tag_layer('morph_analysis')
base_morph_converter.tag(text)

retagger.retag(text)
text['morph_extended']

Layer(name='morph_extended', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'letter_case'), spans=SL[Span('Kumb', [{'normalized_text': 'Kumb', 'lemma': 'kumb', 'root': 'kumb', 'root_tokens': ['kumb'], 'ending': '0', 'clitic': '', 'form': 'sg nom', 'partofspeech': 'P', 'letter_case': 'cap'}]),
Span(',', [{'normalized_text': ',', 'lemma': ',', 'root': ',', 'root_tokens': [','], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z', 'letter_case': None}]),
Span('Sina', [{'normalized_text': 'Sina', 'lemma': 'sina', 'root': 'sina', 'root_tokens': ['sina'], 'ending': '0', 'clitic': '', 'form': 'sg nom', 'partofspeech': 'P', 'letter_case': 'cap'}]),
Span('või', [{'normalized_text': 'või', 'lemma': 'või', 'root': 'või', 'root_tokens': ['või'], 'ending': '0', 'clitic': '', 'form': 'sub crd', 'partofspeech': 'J', 'letter_case': None}]),
Span('mina', [{'normalized_text': 'mina', 'lemma': 'mina', 'root': 'mina', 'root_tokens': ['mina'], 'ending': '0', 'clitic': '', 'form': 'com sg nom', 'partofspeech': 'S', 'letter_case': None}]),
Span('?', [{'normalized_text': '?', 'lemma': '?', 'root': '?', 'root_tokens': ['?'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z', 'letter_case': None}])])

### RemoveAdpositionAnalysesRetagger

RemoveAdpositionAnalysesRetagger removes duplicate adposition analyses (a lexicon specific pre-processing step):

In [6]:
# Convert morph annotations
text = Text('suve eel, talve järel').tag_layer('morph_analysis')
base_morph_converter.tag(text)
# Observe duplicate adposition (K) analyses
text['morph_extended']

Layer(name='morph_extended', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('suve', [{'normalized_text': 'suve', 'lemma': 'suvi', 'root': 'suvi', 'root_tokens': ['suvi'], 'ending': '0', 'clitic': '', 'form': 'com sg gen', 'partofspeech': 'S'}]),
Span('eel', [{'normalized_text': 'eel', 'lemma': 'eel', 'root': 'eel', 'root_tokens': ['eel'], 'ending': '0', 'clitic': '', 'form': 'post', 'partofspeech': 'K'}, {'normalized_text': 'eel', 'lemma': 'eel', 'root': 'eel', 'root_tokens': ['eel'], 'ending': '0', 'clitic': '', 'form': 'pre', 'partofspeech': 'K'}]),
Span(',', [{'normalized_text': ',', 'lemma': ',', 'root': ',', 'root_tokens': [','], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}]),
Span('talve', [{'normalized_text': 'talve', 'lemma': 'talv', 'root': 'talv', 'root_tokens': ['talv'], 'ending': '0', 'clitic': '', 'form': 'com sg gen', 'partofspeech': 'S'}, {'normalized_text': 'talve', 'lemma': 'tali', 'root': 'tali', 'root_tokens': ['tali'], 'ending': '0', 'clitic': '', 'form': 'com sg gen', 'partofspeech': 'S'}]),
Span('järel', [{'normalized_text': 'järel', 'lemma': 'järel', 'root': 'järel', 'root_tokens': ['järel'], 'ending': '0', 'clitic': '', 'form': 'post', 'partofspeech': 'K'}, {'normalized_text': 'järel', 'lemma': 'järel', 'root': 'järel', 'root_tokens': ['järel'], 'ending': '0', 'clitic': '', 'form': 'pre', 'partofspeech': 'K'}])])

In [7]:
from estnltk.taggers.standard.syntax.preprocessing.remove_adposition_analyses_retagger import RemoveAdpositionAnalysesRetagger
retagger = RemoveAdpositionAnalysesRetagger(allow_to_delete_all=False)
# Remove adposition duplicates with the retagger
retagger.retag(text)
text['morph_extended']

Layer(name='morph_extended', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('suve', [{'normalized_text': 'suve', 'lemma': 'suvi', 'root': 'suvi', 'root_tokens': ['suvi'], 'ending': '0', 'clitic': '', 'form': 'com sg gen', 'partofspeech': 'S'}]),
Span('eel', [{'normalized_text': 'eel', 'lemma': 'eel', 'root': 'eel', 'root_tokens': ['eel'], 'ending': '0', 'clitic': '', 'form': 'post', 'partofspeech': 'K'}]),
Span(',', [{'normalized_text': ',', 'lemma': ',', 'root': ',', 'root_tokens': [','], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}]),
Span('talve', [{'normalized_text': 'talve', 'lemma': 'talv', 'root': 'talv', 'root_tokens': ['talv'], 'ending': '0', 'clitic': '', 'form': 'com sg gen', 'partofspeech': 'S'}, {'normalized_text': 'talve', 'lemma': 'tali', 'root': 'tali', 'root_tokens': ['tali'], 'ending': '0', 'clitic': '', 'form': 'com sg gen', 'partofspeech': 'S'}]),
Span('järel', [{'normalized_text': 'järel', 'lemma': 'järel', 'root': 'järel', 'root_tokens': ['järel'], 'ending': '0', 'clitic': '', 'form': 'post', 'partofspeech': 'K'}])])

Flag `allow_to_delete_all` controls, whether RemoveAdpositionAnalysesRetagger is allowed to remove all analyses of a word. 
While the original implementation of syntactic preprocessing allowed this, EstNLTK does not support words/spans without analyses and so this option should be switched off by default. Switch on at your own risk.

### FiniteFormRetagger
FiniteFormRetagger marks whether a verb form is finite or infinite. Non-verb analyses are marked with _None_.

Usage example:

In [8]:
from estnltk.taggers.standard.syntax.preprocessing.finite_form_retagger import FiniteFormRetagger

retagger = FiniteFormRetagger()

text = Text('laulma hüpelnud tantsija').tag_layer('morph_analysis')
base_morph_converter.tag(text)

retagger.retag(text)
text['morph_extended']

Layer(name='morph_extended', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'fin'), spans=SL[Span('laulma', [{'normalized_text': 'laulma', 'lemma': 'laulma', 'root': 'laul', 'root_tokens': ['laul'], 'ending': 'ma', 'clitic': '', 'form': 'mod sup ps ill', 'partofspeech': 'V', 'fin': False}, {'normalized_text': 'laulma', 'lemma': 'laulma', 'root': 'laul', 'root_tokens': ['laul'], 'ending': 'ma', 'clitic': '', 'form': 'aux sup ps ill', 'partofspeech': 'V', 'fin': False}, {'normalized_text': 'laulma', 'lemma': 'laulma', 'root': 'laul', 'root_tokens': ['laul'], 'ending': 'ma', 'clitic': '', 'form': 'main sup ps ill', 'partofspeech': 'V', 'fin': False}]),
Span('hüpelnud', [{'normalized_text': 'hüpelnud', 'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ['hüple'], 'ending': 'nud', 'clitic': '', 'form': 'mod indic impf ps neg', 'partofspeech': 'V', 'fin': True}, {'normalized_text': 'hüpelnud', 'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ['hüple'], 'ending': 'nud', 'clitic': '', 'form': 'mod partic past ps', 'partofspeech': 'V', 'fin': False}, {'normalized_text': 'hüpelnud', 'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ['hüple'], 'ending': 'nud', 'clitic': '', 'form': 'aux indic impf ps neg', 'partofspeech': 'V', 'fin': True}, {'normalized_text': 'hüpelnud', 'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ['hüple'], 'ending': 'nud', 'clitic': '', 'form': 'aux partic past ps', 'partofspeech': 'V', 'fin': False}, {'normalized_text': 'hüpelnud', 'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ['hüple'], 'ending': 'nud', 'clitic': '', 'form': 'main indic impf ps neg', 'partofspeech': 'V', 'fin': True}, {'normalized_text': 'hüpelnud', 'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ['hüple'], 'ending': 'nud', 'clitic': '', 'form': 'main partic past ps', 'partofspeech': 'V', 'fin': False}, {'normalized_text': 'hüpelnud', 'lemma': 'hüpelnud', 'root': 'hüpel=nud', 'root_tokens': ['hüpelnud'], 'ending': '0', 'clitic': '', 'form': 'pos', 'partofspeech': 'A', 'fin': None}, {'normalized_text': 'hüpelnud', 'lemma': 'hüpelnud', 'root': 'hüpel=nud', 'root_tokens': ['hüpelnud'], 'ending': '0', 'clitic': '', 'form': 'pos sg nom', 'partofspeech': 'A', 'fin': None}, {'normalized_text': 'hüpelnud', 'lemma': 'hüpelnud', 'root': 'hüpel=nud', 'root_tokens': ['hüpelnud'], 'ending': 'd', 'clitic': '', 'form': 'pos pl nom', 'partofspeech': 'A', 'fin': None}]),
Span('tantsija', [{'normalized_text': 'tantsija', 'lemma': 'tantsija', 'root': 'tantsija', 'root_tokens': ['tantsija'], 'ending': '0', 'clitic': '', 'form': 'com sg nom', 'partofspeech': 'S', 'fin': None}])])

### VerbExtensionSuffixRetagger
VerbExtensionSuffixRetagger annotates suffixes of (infinite) verb forms. Example:

In [9]:
from estnltk.taggers import VerbExtensionSuffixRetagger

retagger = VerbExtensionSuffixRetagger('morph_extended')

text = Text('Laulev hüpelnud tantsija').tag_layer('morph_analysis')
base_morph_converter.tag(text)

retagger.retag(text)
text['morph_extended']

Layer(name='morph_extended', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'verb_extension_suffix'), spans=SL[Span('Laulev', [{'normalized_text': 'Laulev', 'lemma': 'laulev', 'root': 'laulev', 'root_tokens': ['laulev'], 'ending': '0', 'clitic': '', 'form': 'pos sg nom', 'partofspeech': 'A', 'verb_extension_suffix': ()}]),
Span('hüpelnud', [{'normalized_text': 'hüpelnud', 'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ['hüple'], 'ending': 'nud', 'clitic': '', 'form': 'mod indic impf ps neg', 'partofspeech': 'V', 'verb_extension_suffix': ()}, {'normalized_text': 'hüpelnud', 'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ['hüple'], 'ending': 'nud', 'clitic': '', 'form': 'mod partic past ps', 'partofspeech': 'V', 'verb_extension_suffix': ()}, {'normalized_text': 'hüpelnud', 'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ['hüple'], 'ending': 'nud', 'clitic': '', 'form': 'aux indic impf ps neg', 'partofspeech': 'V', 'verb_extension_suffix': ()}, {'normalized_text': 'hüpelnud', 'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ['hüple'], 'ending': 'nud', 'clitic': '', 'form': 'aux partic past ps', 'partofspeech': 'V', 'verb_extension_suffix': ()}, {'normalized_text': 'hüpelnud', 'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ['hüple'], 'ending': 'nud', 'clitic': '', 'form': 'main indic impf ps neg', 'partofspeech': 'V', 'verb_extension_suffix': ()}, {'normalized_text': 'hüpelnud', 'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ['hüple'], 'ending': 'nud', 'clitic': '', 'form': 'main partic past ps', 'partofspeech': 'V', 'verb_extension_suffix': ()}, {'normalized_text': 'hüpelnud', 'lemma': 'hüpelnud', 'root': 'hüpel=nud', 'root_tokens': ['hüpelnud'], 'ending': '0', 'clitic': '', 'form': 'pos', 'partofspeech': 'A', 'verb_extension_suffix': ('nud',)}, {'normalized_text': 'hüpelnud', 'lemma': 'hüpelnud', 'root': 'hüpel=nud', 'root_tokens': ['hüpelnud'], 'ending': '0', 'clitic': '', 'form': 'pos sg nom', 'partofspeech': 'A', 'verb_extension_suffix': ('nud',)}, {'normalized_text': 'hüpelnud', 'lemma': 'hüpelnud', 'root': 'hüpel=nud', 'root_tokens': ['hüpelnud'], 'ending': 'd', 'clitic': '', 'form': 'pos pl nom', 'partofspeech': 'A', 'verb_extension_suffix': ('nud',)}]),
Span('tantsija', [{'normalized_text': 'tantsija', 'lemma': 'tantsija', 'root': 'tantsija', 'root_tokens': ['tantsija'], 'ending': '0', 'clitic': '', 'form': 'com sg nom', 'partofspeech': 'S', 'verb_extension_suffix': ()}])])

### SubcatRetagger

SubcatRetagger marks verb and adposition subcategorization information, based on [this lexicon / rule file](https://github.com/estnltk/estnltk/blob/main/estnltk/estnltk/taggers/standard/syntax/preprocessing/rules_files/abileksikon06utf.lx).  Usage example:

In [10]:
from estnltk.taggers import SubcatRetagger

retagger = SubcatRetagger()

text = Text('Järel juurduma').tag_layer('morph_analysis')
base_morph_converter.tag(text)

retagger.retag(text)
text['morph_extended']

Layer(name='morph_extended', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'subcat'), spans=SL[Span('Järel', [{'normalized_text': 'Järel', 'lemma': 'järel', 'root': 'järel', 'root_tokens': ['järel'], 'ending': '0', 'clitic': '', 'form': 'post', 'partofspeech': 'K', 'subcat': ('gen',)}, {'normalized_text': 'Järel', 'lemma': 'järel', 'root': 'järel', 'root_tokens': ['järel'], 'ending': '0', 'clitic': '', 'form': 'pre', 'partofspeech': 'K', 'subcat': None}]),
Span('juurduma', [{'normalized_text': 'juurduma', 'lemma': 'juurduma', 'root': 'juurdu', 'root_tokens': ['juurdu'], 'ending': 'ma', 'clitic': '', 'form': 'mod sup ps ill', 'partofspeech': 'V', 'subcat': ('Intr',)}, {'normalized_text': 'juurduma', 'lemma': 'juurduma', 'root': 'juurdu', 'root_tokens': ['juurdu'], 'ending': 'ma', 'clitic': '', 'form': 'aux sup ps ill', 'partofspeech': 'V', 'subcat': ('Intr',)}, {'normalized_text': 'juurduma', 'lemma': 'juurduma', 'root': 'juurdu', 'root_tokens': ['juurdu'], 'ending': 'ma', 'clitic': '', 'form': 'main sup ps ill', 'partofspeech': 'V', 'subcat': ('Intr',)}])])

### MorphExtendedTagger

MorphExtendedTagger glues together all forementioned taggers and retaggers into a single preprocessing tagger, which produces the 'morph_extended' layer.

In [11]:
from estnltk.taggers import MorphExtendedTagger

tagger = MorphExtendedTagger()
tagger

MorphExtendedTagger(input_layers=('morph_analysis',), output_layer=morph_extended, output_attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form',  ..., type: <class 'tuple'>, length: 14, punctuation_type_retagger=PunctuationTypeRetagger(('morph_extended',)->morph_extended), morph_to_syntax_morph_retagger=MorphToSyntaxMorphRetagger(('morph_analysis',)->morph_extended), pronoun_type_retagger=PronounTypeRetagger(('morph_extended',)->morph_extended), letter_case_retagger=LetterCaseRetagger(('morph_extended',)->morph_extended), remove_adposition_analyses_retagger=RemoveAdpositionAnalysesRetagger(('morph_extended',)->morph_extended), finite_form_retagger=FiniteFormRetagger(('morph_extended',)->morph_extended), verb_extension_suffix_retagger=VerbExtensionSuffixRetagger(('morph_extended',)->morph_extended), subcat_retagger=SubcatRetagger(('morph_extended',)->morph_extended))

In [12]:
text = Text('Ta on rääkinud!').tag_layer()
tagger.tag(text)
text['morph_extended']

Layer(name='morph_extended', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'punctuation_type', 'pronoun_type', 'letter_case', 'fin', 'verb_extension_suffix', 'subcat'), spans=SL[Span('Ta', [{'normalized_text': 'Ta', 'lemma': 'tema', 'root': 'tema', 'root_tokens': ['tema'], 'ending': '0', 'clitic': '', 'form': 'sg nom', 'partofspeech': 'P', 'punctuation_type': None, 'pronoun_type': ['ps3'], 'letter_case': 'cap', 'fin': None, 'verb_extension_suffix': [], 'subcat': None}]),
Span('on', [{'normalized_text': 'on', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'mod indic pres ps3 sg ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': [], 'subcat': ['Intr']}, {'normalized_text': 'on', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'aux indic pres ps3 sg ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': [], 'subcat': ['Intr']}, {'normalized_text': 'on', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'main indic pres ps3 sg ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': [], 'subcat': ['Intr']}, {'normalized_text': 'on', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'mod indic pres ps3 pl ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': [], 'subcat': ['Intr']}, {'normalized_text': 'on', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'aux indic pres ps3 pl ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': [], 'subcat': ['Intr']}, {'normalized_text': 'on', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'main indic pres ps3 pl ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': [], 'subcat': ['Intr']}]),
Span('rääkinud', [{'normalized_text': 'rääkinud', 'lemma': 'rääkima', 'root': 'rääki', 'root_tokens': ['rääki'], 'ending': 'nud', 'clitic': '', 'form': 'mod indic impf ps neg', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': [], 'subcat': ['Part-P', 'El']}, {'normalized_text': 'rääkinud', 'lemma': 'rääkima', 'root': 'rääki', 'root_tokens': ['rääki'], 'ending': 'nud', 'clitic': '', 'form': 'mod partic past ps', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': False, 'verb_extension_suffix': [], 'subcat': ['Part-P', 'El']}, {'normalized_text': 'rääkinud', 'lemma': 'rääkima', 'root': 'rääki', 'root_tokens': ['rääki'], 'ending': 'nud', 'clitic': '', 'form': 'aux indic impf ps neg', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': [], 'subcat': ['Part-P', 'El']}, {'normalized_text': 'rääkinud', 'lemma': 'rääkima', 'root': 'rääki', 'root_tokens': ['rääki'], 'ending': 'nud', 'clitic': '', 'form': 'aux partic past ps', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': False, 'verb_extension_suffix': [], 'subcat': ['Part-P', 'El']}, {'normalized_text': 'rääkinud', 'lemma': 'rääkima', 'root': 'rääki', 'root_tokens': ['rääki'], 'ending': 'nud', 'clitic': '', 'form': 'main indic impf ps neg', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': [], 'subcat': ['Part-P', 'El']}, {'normal

## CG3 exporter

CG3 exporter can be used to export Text with 'morph_extended' layer into the [Visl CG format](https://visl.sdu.dk/cg3/single/#stream-vislcg). Example:

In [13]:
from estnltk.converters import export_CG3
text = Text('Lähme! Ta on rääkinud.')
text.tag_layer(['sentences','morph_extended'])
export_CG3(text)

['"<s>"',
 '"<Lähme>"',
 '    "mine" Lme V mod indic pres ps1 pl ps af cap <FinV>',
 '    "mine" Lme V aux indic pres ps1 pl ps af cap <FinV>',
 '    "mine" Lme V main indic pres ps1 pl ps af cap <FinV>',
 '"<!>"',
 '    "!" Z Exc',
 '"</s>"',
 '"<s>"',
 '"<Ta>"',
 '    "tema" L0 P pers ps3 sg nom cap',
 '"<on>"',
 '    "ole" L0 V mod indic pres ps3 sg ps af <FinV> <Intr>',
 '    "ole" L0 V aux indic pres ps3 sg ps af <FinV> <Intr>',
 '    "ole" L0 V main indic pres ps3 sg ps af <FinV> <Intr>',
 '    "ole" L0 V mod indic pres ps3 pl ps af <FinV> <Intr>',
 '    "ole" L0 V aux indic pres ps3 pl ps af <FinV> <Intr>',
 '    "ole" L0 V main indic pres ps3 pl ps af <FinV> <Intr>',
 '"<rääkinud>"',
 '    "rääki" Lnud V mod indic impf ps neg <FinV> <Part-P> <El>',
 '    "rääki" Lnud V mod partic past ps <Part-P> <El>',
 '    "rääki" Lnud V aux indic impf ps neg <FinV> <Part-P> <El>',
 '    "rääki" Lnud V aux partic past ps <Part-P> <El>',
 '    "rääki" Lnud V main indic impf ps neg <FinV> <Par

This is useful when we want to test Visl CG3 parser without the EstNLTK's interface.

## ConllMorphTagger and 'conll_morph' layer

At its simplest, the 'conll_morph' layer is a representation of morpho-syntactic information using fields of the [CoNLL format](https://universaldependencies.org/format.html).
In this way, both 'morph_analysis' and 'morph_extended' layers can be converted to a 'conll_morph' layer ( which has fields _id, form, lemma, upostag, xpostag, feats, head, deprel, deps, misc_ ), while keeping their original morphological categories.
In a more complex pre-processing, 'morph_extended' layer can be processed with Visl CG3-based syntactic parser, which produces a variant of 'conll_morph' layer with extended categories.

The 'conll_morph' layer is used as an input by MaltParser's and UDPipe's models.

In the following, we'll exemplify the usages of ConllMorphTagger.

### ConllMorphTagger on 'morph_analysis' layer

ConllMorphTagger can be used to convert Vabamorf's 'morph_analysis' attributes to CONLL-U attributes (without changing the categories):

In [14]:
from estnltk import Text
from estnltk.taggers import ConllMorphTagger

text = Text("Ta on rääkinud.").tag_layer('morph_analysis')

conll_morph = ConllMorphTagger( morph_extended_layer='morph_analysis',
                                no_visl=True )
conll_morph.tag(text)
text['conll_morph']

Layer(name='conll_morph', attributes=('id', 'form', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc'), spans=SL[Span('Ta', [{'id': 1, 'form': 'Ta', 'lemma': 'tema', 'upostag': 'P', 'xpostag': 'P', 'feats': 'sg|n', 'head': '_', 'deprel': '_', 'deps': '_', 'misc': '_'}]),
Span('on', [{'id': 2, 'form': 'on', 'lemma': 'olema', 'upostag': 'V', 'xpostag': 'V', 'feats': 'b', 'head': '_', 'deprel': '_', 'deps': '_', 'misc': '_'}]),
Span('rääkinud', [{'id': 3, 'form': 'rääkinud', 'lemma': 'rääkima', 'upostag': 'V', 'xpostag': 'V', 'feats': 'pl|n', 'head': '_', 'deprel': '_', 'deps': '_', 'misc': '_'}]),
Span('.', [{'id': 4, 'form': '.', 'lemma': '.', 'upostag': 'Z', 'xpostag': 'Z', 'feats': '_', 'head': '_', 'deprel': '_', 'deps': '_', 'misc': '_'}])])

*Note:* 'conll_morph' does not have ambiguous annotations. This is a result of "random pick disambiguation strategy", which always picks the first part of speech and random _form_ in case of an ambiguity.

### ConllMorphTagger on 'morph_extended' layer (without Visl CG3 parsing)

ConllMorphTagger can be used to convert 'morph_extended' attributes to CONLL-U attributes (without changing the categories):

In [15]:
from estnltk import Text
from estnltk.taggers import ConllMorphTagger

text = Text("Ta on rääkinud.").tag_layer('morph_extended')

conll_morph = ConllMorphTagger( morph_extended_layer='morph_extended',
                                no_visl=True )
conll_morph.tag(text)
text['conll_morph']

Layer(name='conll_morph', attributes=('id', 'form', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc'), spans=SL[Span('Ta', [{'id': 1, 'form': 'Ta', 'lemma': 'tema', 'upostag': 'P', 'xpostag': 'P', 'feats': 'sg|nom', 'head': '_', 'deprel': '_', 'deps': '_', 'misc': '_'}]),
Span('on', [{'id': 2, 'form': 'on', 'lemma': 'olema', 'upostag': 'V', 'xpostag': 'V', 'feats': 'aux|indic|pres|ps3|sg|ps|af', 'head': '_', 'deprel': '_', 'deps': '_', 'misc': '_'}]),
Span('rääkinud', [{'id': 3, 'form': 'rääkinud', 'lemma': 'rääkima', 'upostag': 'V', 'xpostag': 'V', 'feats': 'pos|sg|nom', 'head': '_', 'deprel': '_', 'deps': '_', 'misc': '_'}]),
Span('.', [{'id': 4, 'form': '.', 'lemma': '.', 'upostag': 'Z', 'xpostag': 'Z', 'feats': '_', 'head': '_', 'deprel': '_', 'deps': '_', 'misc': '_'}])])

*Note:* Again, 'conll_morph' layer does not have ambiguous annotations, but this is a result of a random pick from ambiguous forms and keeping the first part of speech on each ambiguous word.

### ConllMorphTagger on 'morph_extended' layer (with Visl CG3 parsing)

ConllMorphTagger can be used to process 'morph_extended' layer with VislCG3 parser's Estonian rules, which involves rule-based morphological disambiguation and updating morphological categories for syntax.

Note: this requires that the **VISLCG3 parser is installed into the system, and accessible from system's environment variable PATH** . The information about the parser is distributed in the [Constraint Grammar's Google Group](https://groups.google.com/g/constraint-grammar), and this is also the place to look for the most compact guide about [getting & installing the parser](https://groups.google.com/g/constraint-grammar/c/fNMkpAb_g3U).

You can check the availability of the VISLCG3 parser by typing:

In [16]:
!vislcg3 -V

VISL CG-3 Disambiguator version 1.3.7.13892
Copyright (C) 2007-2021 GrammarSoft ApS. Licensed under GPLv3+


Once you have the parser installed and available, you can apply ConllMorphTagger to produce an enchanced 'conll_morph' layer:

In [17]:
from estnltk import Text
from estnltk.taggers import ConllMorphTagger

text = Text("Ta on rääkinud.").tag_layer('morph_extended')
text.tag_layer('morph_extended')

conll_morph = ConllMorphTagger(morph_extended_layer='morph_extended')
conll_morph.tag(text)
text['conll_morph']

Layer(name='conll_morph', attributes=('id', 'form', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc'), spans=SL[Span('Ta', [{'id': 1, 'form': 'Ta', 'lemma': 'tema', 'upostag': 'P', 'xpostag': 'Ppers', 'feats': 'ps3|sg|nom', 'head': '_', 'deprel': '_', 'deps': '_', 'misc': '_'}]),
Span('on', [{'id': 2, 'form': 'on', 'lemma': 'ole', 'upostag': 'V', 'xpostag': 'Vaux', 'feats': 'indic|pres|ps3|sg', 'head': '_', 'deprel': '_', 'deps': '_', 'misc': '_'}]),
Span('rääkinud', [{'id': 3, 'form': 'rääkinud', 'lemma': 'rääki', 'upostag': 'V', 'xpostag': 'V', 'feats': 'ppast', 'head': '_', 'deprel': '_', 'deps': '_', 'misc': '_'}]),
Span('.', [{'id': 4, 'form': '.', 'lemma': '.', 'upostag': 'Z', 'xpostag': 'Z', 'feats': 'Fst', 'head': '_', 'deprel': '_', 'deps': '_', 'misc': '_'}])])

## CONLL exporter

Text with 'conll_morph' layer can also be exported into the [CoNLL format](https://universaldependencies.org/format.html) string. Example:

In [18]:
from estnltk.taggers.standard.syntax.conll_morph_to_str import conll_to_str

from estnltk import Text
from estnltk.taggers import ConllMorphTagger

text = Text("Lähme! Ta on rääkinud.").tag_layer('morph_extended')

conll_morph = ConllMorphTagger( morph_extended_layer='morph_extended',
                                no_visl=True )
conll_morph.tag(text)

print(conll_to_str(text))

1	Lähme	minema	V	V	aux|indic|pres|ps1|pl|ps|af	_	_	_	_	
2	!	!	Z	Z	_	_	_	_	_	

1	Ta	tema	P	P	sg|nom	_	_	_	_	
2	on	olema	V	V	main|indic|pres|ps3|pl|ps|af	_	_	_	_	
3	rääkinud	rääkima	V	V	mod|indic|impf|ps|neg	_	_	_	_	
4	.	.	Z	Z	_	_	_	_	_	




This is useful when we want to test Maltparser or UDPipe without the EstNLTK's interface.